In [1]:
!pip install datasets
from tokenizers import (decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer)
from transformers import GPT2Tokenizer, GPT2TokenizerFast, GPT2Model, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from huggingface_hub import login
import logging
import time
from datasets import Dataset, DatasetDict, load_from_disk
import os
import torch

In [2]:
# Hugging Face Token
hf_token='XXXX'
login(token=hf_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Load the fine-tuned GPT-2 model for inference
tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/Study/extended_tokenizer_gpt2")
tokenizer.pad_token = tokenizer.eos_token
finetuned_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Study/rus_gpt2_2_epoch")

finetuned_model.eval()

# Input text for completion
input_text = "Однажды она решила"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")



max_length = 250
# # Generate text completions
# output_text = finetuned_model.generate(input_ids, max_length=max_length)[0]

output_text = finetuned_model.generate(
    input_ids,
    max_length=max_length,
    temperature=0.8,              # Controls the diversity of the generated text
    top_k=50,                     # Keeps only the top-k most likely words
    top_p=0.9,                    # Nucleus sampling (cumulative probability)
    repetition_penalty=1.2,       # Penalty for repeating words or phrases
    no_repeat_ngram_size=4,       # Prevents repetition of n-grams (e.g., bigrams)
    do_sample=True,                # Enables sampling for greater diversity
    pad_token_id=tokenizer.eos_token_id
)[0]

#Decode the generated token IDs to text
completed_text = tokenizer.decode(output_text, skip_special_tokens=True, clean_up_tokenization_spaces=True)


print("Input Text:", input_text)
print("Completed Text:", completed_text)

Input Text: Однажды она решила
Completed Text: Однажды она решила отправиться в город на поиски своего друга и помощника, который предлагает ему вернуться обратно. Но на этот раз ей удаётся уйти из города и забрать его с собой, чтобы спасти её. Тогда Стиву приходится поговорить со своей подругой Розиной (Нью-Йорк), но она понимает, что не может проявлять себя как мистер Мелвин. Кристин останавливается и говорит, что это просто та же история о том, кто живет. Она знакомит девушку с молодого Стива, которая начинает с нейтвы, поскольку они делают своих детей. После того жесто лететь он попадает за то, когда она уже давит их же самую, а потом. Тем, потому что они пытаются другому, если убивают вместе с помощью ещё сильнее, а затем будут выгляд, так называемые.   Дже. Когда она любит себя любовница сделала свою дочь, пока он хотела у неё ещё ранься, что она сама. Ната.  вшуюся. В то есть только тогда ещё


In [ ]:
#Further training!

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Devive is: {device}")

# Load the  model
tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/Study/extended_tokenizer_gpt2")
tokenizer.pad_token = tokenizer.eos_token
finetuned_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Study/rus_gpt2_2_epoch").to(device)

# Dataset
lm_dataset = load_from_disk('/content/drive/MyDrive/Study/lm_dataset')

tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/Study/extended_tokenizer_gpt2")
# Model to CUDA
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Study/rus_gpt2_2_epoch").to(device)

# Setting up training arguments
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Study/rus_gpt2_3_epoch_dop/",
    overwrite_output_dir=True,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="steps",
    eval_steps=2000,
    logging_steps=2000,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=200,
    lr_scheduler_type="cosine",
    learning_rate=5e-3,
    save_steps=10000,
    save_total_limit=2,
    #push_to_hub=True,
    fp16=True,

)

#  Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["valid"],
)


trainer.train()

# Save
trainer.save_model("/content/drive/MyDrive/Study/rus_gpt2_3_epoch_dop")

